In [19]:
import time

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, IntegerType, FloatType, BooleanType

In [20]:
spark = SparkSession.builder.getOrCreate() # Initialize Spark session

In [21]:
data_schema = StructType(
    [
        # StructField("$schema", StringType(), True),
        # StructField(
        #     "meta",
        #     StructType([
        #         StructField("uri", StringType(), True),
        #         StructField("request_id", StringType(), True),
        #         StructField("id", StringType(), True),
        #         StructField("dt", StringType(), True),
        #         StructField("domain", StringType(), True),
        #         StructField("stream", StringType(), True),
        #         StructField("topic", StringType(), True),
        #         StructField("partition", StringType(), True),
        #         StructField("offset", StringType(), True)
        #     ]),
        #     True
        # ),
        # StructField("id", StringType(), True),
        # StructField("type", StringType(), True),
        # StructField("namespace", StringType(), True),
        StructField("title", StringType(), True),
        StructField("title_url", StringType(), True),
        StructField("comment", StringType(), True),
        #StructField("timestamp", StringType(), True),
        StructField("user", StringType(), True),
        StructField("bot", StringType(), True),
        #StructField("notify_url", StringType(), True),
        #StructField("minor", StringType(), True),
        #StructField("patrolled", StringType(), True),
        # StructField(
        #     "length",
        #     StructType([
        #         StructField("old", StringType(), True),
        #         StructField("new", StringType(), True)
        #     ]),
        #     True
        # ),
        # StructField(
        #     "revision",
        #     StructType([
        #         StructField("old", StringType(), True),
        #         StructField("new", StringType(), True)
        #     ]),
        #     True
        # ),
        # StructField("server_url", StringType(), True),
        # StructField("server_name", StringType(), True),
        # StructField("server_script_path", StringType(), True),
        #StructField("wiki", StringType(), True),
        StructField("parsedcomment", StringType(), True)
    ]
)

In [25]:
# Read streaming data from socket
stream_df = (
    spark.readStream
    .format("socket")
    .option("host", "localhost")
    .option("port", 9999)
    .load()
)

# stream_df = (
#     spark.readStream.format("json")
#     .option("path", "stream")
#     .schema(data_schema)
#     .load()
# )

24/11/17 18:01:38 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


In [26]:
stream_df = (
    stream_df.filter(F.col("value").startswith("data: {"))
    .select(F.trim(F.substring(F.col("value"), 7, 1_000_000)).alias("json_string"))
    .select(F.from_json(F.col("json_string"), data_schema).alias("parsed"))
    .select("parsed.*")
)

In [27]:
stream_df = spark.readStream.format("rate").option("rowsPerSecond", 3).load()
query = (
    stream_df.writeStream
    .trigger(processingTime="1 seconds")
    .outputMode("append")
    .format("console")
    .start()
)
time.sleep(5)
query.stop()

24/11/17 18:01:39 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-b9005cd6-1f66-4529-9ac5-555f7174a88d. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/11/17 18:01:39 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+-----+---------+-------+----+---+-------------+
|title|title_url|comment|user|bot|parsedcomment|
+-----+---------+-------+----+---+-------------+
+-----+---------+-------+----+---+-------------+

-------------------------------------------
Batch: 1
-------------------------------------------
+-------------------------+--------------------+--------------------+---------------+-----+--------------------+
|                    title|           title_url|             comment|           user|  bot|       parsedcomment|
+-------------------------+--------------------+--------------------+---------------+-----+--------------------+
|     Category:WikiProj...|https://en.wikipe...|[[:Talk:Zach Wils...|       SporkBot| true|<a href="/wiki/Ta...|
|     Category:Creative...|https://commons.w...|[[:File:Vuelo del...|      Luisalvaz|false|<a href="/wiki/Fi...|
|               Q131250799|https://www.wikid